# Capstone 2 Pre-processing<a id="Pre-processing"></a>

note: will to look at .info() and change some datatypes... gender is a float rn. 

note: could create a "seconds per day" column

## Table of Contents<a id='Table_of_Contents'></a>
* [3 Pre-processing](#Pre-processing)
    * [3.1 Imports](#Imports)
        * [3.1.1 Import Libraries](#Import_Libraries)
        * [3.1.2 Import Data](#Import_Data)          
    
    * [3.2 ](#)
        * [3.2.1 ](#)
        
    * [3.3 ](#)
    
    * [3.4 ](#)

## 3.1 Imports<a id="Imports"></a>

### 3.1.1 Import Libraries<a id="Import_Libraries"></a>

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

### 3.1.2 Import Data<a id="Import_Data"></a>

In [3]:
data = pd.read_csv("../data/processed/combined.csv")

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 599990 entries, 0 to 599989
Data columns (total 24 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Unnamed: 0              599990 non-null  int64  
 1   msno                    599990 non-null  object 
 2   transaction_date        599990 non-null  int64  
 3   is_churn                599990 non-null  int64  
 4   payment_method_id       599990 non-null  int64  
 5   payment_plan_days       599990 non-null  int64  
 6   plan_list_price         599990 non-null  int64  
 7   actual_amount_paid      599990 non-null  int64  
 8   is_auto_renew           599990 non-null  int64  
 9   membership_expire_date  599990 non-null  int64  
 10  is_cancel               599990 non-null  int64  
 11  city                    599990 non-null  int64  
 12  age                     599990 non-null  int64  
 13  gender                  599990 non-null  float64
 14  registered_via      

In [ ]:
# convert gender column to int64


In [ ]:
data.head()

In [ ]:
#X_train.to_csv("../data/processed/X_train.csv")
#y_train.to_csv("../data/processed/y_train.csv")
#X_test.to_csv("../data/processed/X_test.csv")
#y_test.to_csv("../data/processed/y_test.csv")